In [1]:
import tensorflow as tf
import numpy as np

from tensorflow.keras.datasets import mnist

In this example, the MNIST dataset will be used that is packaged as part of the TensorFlow
installation. This MNIST dataset is a set of 28×28 pixel grayscale images which represent
hand-written digits. It has 60,000 training rows, 10,000 testing rows and 5,000 validation
rows. It is a very common, basic, image classification dataset that is used in machine
learning. 

In [2]:
tf.__version__

'2.0.0'

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [4]:
def get_batch(x, y, size):
    idxs = np.random.randint(0, len(y), size)
    return x[idxs,:,:], y[idxs]

In [5]:
epochs = 10
batch_size = 100

# normaliza as imagens, valores ficam entre 0 e 1
x_train = x_train / 255.0
x_test = x_test / 255.0

# converte o x_test para tensor para ser usado no modelo, dados de treinamento serão convertidos on the fly
x_test = tf.Variable(x_test)

## Cria os layers 

In [6]:
# Declara pesos conectando o input com a camada oculta
W1 = tf.Variable(tf.random.normal([784, 300], stddev= 0.03), name='w1')
b1 = tf.Variable(tf.random.normal([300]), name='b1')

# e os pesos conectando a camada oculta à camada de saida
W2 = tf.Variable(tf.random.normal([300, 10], stddev= 0.03), name='w2')
b2 = tf.Variable(tf.random.normal([10]), name='b2')

O formato do W1 é um tensor [784,300] - os 784 nós é o tamanho da camada de entrada, como nós temos imagens de 28x28. multiplicando esses valores <br>
temos 784, os 300 são o numéro de nós na camada oculta. O W2 é um tensor [300,10], que conecta os 300 nós da camada oculta <br>
aos 10 nós da camada de saída, que são o número de classes do dataset 

In [7]:
def nn_model(x, W1, b1, W2, b2):
    # deiaa o arra flat, da imagem de 28x28 se torna 784
    x = tf.reshape(x, (x.shape[0], -1))
    x = tf.add(tf.matmul(tf.cast(x , tf.float32), W1), b1)
    x = tf.nn.relu(x)
    
    logits = tf.add(tf.matmul(x, W2), b2)
    return logits

![title](images/nn_model.png) <br>

onde W são os pesos da matriz, x é o vetor de entrada, b é o bias e f é a função de ativação. Esse cálculo é para a operação de feed-foward

In [8]:
def loss_fn(logits, labels):
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = labels, 
                                                                           logits= logits))
    return cross_entropy

In [9]:
# cria otimizador
optimizer = tf.keras.optimizers.Adam()

## Treinando a rede

In [10]:
# determina o numero de batches para roda em cada época, garante que na média cada amostra de treinamento será usada
total_batch = int(len(y_train)/ batch_size)

for epoch in range(epochs):
    
    # acompanha a média de loss para cada época
    avg_loss = 0
    
    for i in range(total_batch):
        
        # obtem uma amostra aleatória
        batch_x, batch_y = get_batch(x_train, y_train, batch_size)
        
        # Cria tensores
        batch_x = tf.Variable(batch_x)
        batch_y = tf.Variable(batch_y)
        
        # Cria vetor one hot
        batch_y = tf.one_hot(batch_y, 10)
        
        # API para o TF calcular os gradients, qualquer operação e/ou variveis dentro desse contexto, o TF irá capturar os gradientes
        with tf.GradientTape() as tape:
            
            # TF agora sabe quais operações e variaveis deve fazer o tracking, para  depois podermos fazer operações com os gradientes
            logits = nn_model(batch_x, W1, b1, W2, b2)
            loss = loss_fn(logits, batch_y)
         
        #calcula a derivada (dL/dw & Dl/db)
        gradients = tape.gradient(loss, [W1, b1, W2, b2])
        
        # associa o gradient aos pesos e bias e executa o gradiente descendente
        optimizer.apply_gradients(zip(gradients, [W1, b1, W2, b2]))
        
        # Loss médio da época
        avg_loss += loss / total_batch
        
    test_logits = nn_model(x_test, W1, b1, W2, b2)
    max_idxs = tf.argmax(test_logits, axis= 1)
    test_acc = np.sum(max_idxs.numpy() == y_test)/ len(y_test)
    
    print(f"Epoch:{epoch + 1}, loss={avg_loss:.3f}, test set    accuracy={test_acc * 100:.3f}%")

print("\nTraining complete")
        
            
        

Epoch:1, loss=0.367, test set    accuracy=93.840%
Epoch:2, loss=0.155, test set    accuracy=96.180%
Epoch:3, loss=0.100, test set    accuracy=96.800%
Epoch:4, loss=0.075, test set    accuracy=97.360%
Epoch:5, loss=0.059, test set    accuracy=97.600%
Epoch:6, loss=0.045, test set    accuracy=97.620%
Epoch:7, loss=0.035, test set    accuracy=97.850%
Epoch:8, loss=0.030, test set    accuracy=97.750%
Epoch:9, loss=0.022, test set    accuracy=97.680%
Epoch:10, loss=0.019, test set    accuracy=97.990%

Training complete
